In [1]:
# Data Handling & Manipulation
import pandas as pd                                         # for dataframes, CSV/Excel reading, tabular data manipulation
import numpy as np                                          # for numerical operations and array handling
import datetime                                             # for date and time manipulation
from datetime import datetime, timedelta                    # for date arithmetic (e.g., adding days)
from workalendar.america import Brazil                      # for Brazilian holidays (e.g., to check if a date is a holiday)

# Data Visualisation (Static)
import matplotlib.pyplot as plt                             # for creating static plots
from matplotlib.ticker import FuncFormatter                 # for customising tick labels (e.g., currency, %)
import seaborn as sns                                       # for statistical visualisation (heatmaps, distplots, etc.)

# Data Visualisation (Interactive)
import plotly.express as px                                 # for quick and interactive visualisation
import plotly.graph_objects as go                           # for custom interactive plots
from plotly.subplots import make_subplots                   # for interactive subplots

# Data Quality & Missing Value Visualisation
import missingno as msno                                    # for visualising missing data patterns

# Statistical Testing & Inference
from statsmodels.stats.proportion import proportions_ztest  # for comparing proportions (e.g., late vs. on-time)
from scipy.stats import (
    normaltest,                                             # for checking normality
    chi2_contingency,                                       # for categorical association
    mannwhitneyu,                                           # for non-parametric testing
    ttest_ind,                                              # for independent sample t-test
    f_oneway,                                               # for one-way ANOVA
    kruskal,                                                # for Kruskal-Wallis test
    kstest,                                                 # for Kolmogorov-Smirnov test
    spearmanr,                                              # for Spearman correlation
    pointbiserialr                                          # for point-biserial correlation (binary vs. continuous)
)
import statsmodels.api as sm                                # for advanced statistical modelling and diagnostics
import statsmodels.formula.api as smf                       # for formula-based statistical models
from statsmodels.stats.multicomp import pairwise_tukeyhsd   # for post-hoc tests after ANOVA

# System & Settings
import os                                                   # for file handling and directory operations
import warnings                                             # to suppress or manage warning messages
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_colwidth', None)                 # display full content in cells (useful for text data)

# Machine Learning & Model Evaluation
import joblib                                               # for saving and loading ML models
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    average_precision_score, precision_recall_fscore_support,
    confusion_matrix, PrecisionRecallDisplay, make_scorer
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier           # for KNN classifier (added since it's in your benchmark)
from sklearn.tree import DecisionTreeClassifier              # for Decision Tree classifier (added since it's in your benchmark)
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBClassifier                           # for XGBoost classifier
from lightgbm import LGBMClassifier                         # for LightGBM classifier
from catboost import CatBoostClassifier, Pool               # for CatBoost classifier


In [2]:
# List of date columns for each Olist dataset:
# This dictionary maps each dataset filename to a list of columns that should be parsed as dates.
date_cols = {
    'olist_orders_dataset.csv': [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
    ],
    'olist_order_items_dataset.csv': [
        'shipping_limit_date',
    ],
    'olist_order_reviews_dataset.csv': [
        'review_creation_date',
        'review_answer_timestamp',
    ],
    # The following datasets have NO date columns:
    # 'olist_customers_dataset.csv'
    # 'olist_geolocation_dataset.csv'
    # 'olist_order_payments_dataset.csv'
    # 'olist_products_dataset.csv'
    # 'olist_sellers_dataset.csv'
    # 'product_category_name_translation.csv'
    'master_olist_dataset.csv': [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
        'shipping_limit_date',
        'review_creation_date',
        'review_answer_timestamp',
    ],
}

def read_olist_csv(path):
    """
    Reads an Olist CSV and parses dates for the correct columns.
    Args:
        path (str): Path to the CSV file.
    Returns:
        pd.DataFrame: Loaded dataframe with date columns parsed as datetime.
    """
    # Extract just the filename, e.g., 'olist_orders_dataset.csv':
    filename = os.path.basename(path)
    # Get the correct date columns for this file, or an empty list:
    parse_dates = date_cols.get(filename, [])
    # Read the CSV, parsing the specified date columns (if any):
    return pd.read_csv(path, parse_dates=parse_dates)

In [3]:
df = read_olist_csv('../data/cleaned_data/olist_ml_ready_dataset.csv')
df.head()

,purchase_to_approve_hrs,approve_to_estimated_days,approve_to_shipping_limit_days,purchase_hour,purchase_dow,purchase_month,is_weekend,is_brazil_holiday,distance_km,same_state,...,total_order_lifetime,sum_freight_value,price,total_payment_value,customer_state,seller_state,freight_value,product_category_name_english,payment_types,is_late
0,0.178333,15,4,10,0,10,0,0,18.657513,1,...,53,8.72,29.99,38.71,SP,SP,8.72,housewares,"credit_card, voucher",False
1,30.713889,17,4,20,1,7,0,0,861.068703,0,...,125,22.76,118.70,141.46,BA,SP,22.76,perfumery,boleto,False
2,0.276111,26,5,8,2,8,0,0,514.560686,0,...,1148,19.22,159.90,179.12,GO,SP,19.22,auto,credit_card,False
3,0.298056,26,5,19,5,11,1,0,1821.871635,0,...,155,27.20,45.00,72.20,RN,MG,27.20,pet_shop,credit_card,False
4,1.030556,12,5,21,1,2,0,0,29.623876,1,...,171,8.72,19.90,28.62,SP,SP,8.72,stationery,credit_card,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109933 entries, 0 to 109932
Data columns (total 29 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   purchase_to_approve_hrs                         109933 non-null  float64
 1   approve_to_estimated_days                       109933 non-null  int64  
 2   approve_to_shipping_limit_days                  109933 non-null  int64  
 3   purchase_hour                                   109933 non-null  int64  
 4   purchase_dow                                    109933 non-null  int64  
 5   purchase_month                                  109933 non-null  int64  
 6   is_weekend                                      109933 non-null  int64  
 7   is_brazil_holiday                               109933 non-null  int64  
 8   distance_km                                     109933 non-null  float64
 9   same_state                

Change the target variable (`is_late`) to an int for the model training

In [5]:
df['is_late'] = df['is_late'].astype(int)

Numerical features descriptive statistics:

In [6]:
# Set pandas to display all columns and rows
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)
# pd.set_option('display.width', 1000)

# Get descriptive statistics for numerical features
df.describe()

,purchase_to_approve_hrs,approve_to_estimated_days,approve_to_shipping_limit_days,purchase_hour,purchase_dow,purchase_month,is_weekend,is_brazil_holiday,distance_km,same_state,freight_ratio,customer_is_remote,seller_dispatch_hub,seller_30d_late_rate_is_dispatch_late_raw,seller_30d_late_rate_is_dispatch_late_smoothed,seller_30d_order_count,seller_90d_late_rate_is_dispatch_late_raw,seller_90d_late_rate_is_dispatch_late_smoothed,seller_90d_order_count,total_order_lifetime,sum_freight_value,price,total_payment_value,freight_value,is_late
count,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.00000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000,109933.000000
mean,10.533222,22.912974,5.985355,14.751258,2.746537,6.030701,0.227302,0.027699,596.771304,0.361820,0.320681,0.278088,0.627820,0.042141,0.123733,35.309880,0.044501,0.095948,90.02809,422.691303,27.248761,120.044161,179.509910,19.949499,0.079212
std,21.000989,8.832961,5.467422,5.319016,1.963767,3.233580,0.419091,0.164109,587.318831,0.480529,0.342008,0.448059,0.160439,0.129461,0.124669,49.067543,0.115396,0.114392,127.36208,556.063689,33.270072,182.449223,271.586656,15.701392,0.270070
min,0.000000,-7.000000,-7.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074074,0.000000,0.002604,1.000000,0.000000,0.001502,1.00000,1.000000,0.000000,0.850000,9.590000,0.000000,0.000000
25%,0.216389,17.000000,4.000000,11.000000,1.000000,3.000000,0.000000,0.000000,185.951083,0.000000,0.134536,0.000000,0.515419,0.000000,0.037037,6.000000,0.000000,0.023158,12.00000,57.000000,14.290000,39.900000,65.550000,13.080000,0.000000
50%,0.350556,22.000000,6.000000,15.000000,3.000000,6.000000,0.000000,0.000000,433.348916,0.000000,0.231806,0.000000,0.601190,0.000000,0.083333,15.000000,0.000000,0.054054,35.00000,171.000000,18.160000,74.900000,114.340000,16.260000,0.000000
75%,15.194722,28.000000,6.000000,19.000000,4.000000,8.000000,0.000000,0.000000,793.859072,1.000000,0.393229,1.000000,0.732713,0.012987,0.166667,42.000000,0.035714,0.125000,105.00000,521.000000,29.170000,134.500000,194.910000,21.150000,0.000000
max,741.443611,153.000000,1051.000000,23.000000,6.000000,12.000000,1.000000,1.000000,3398.548220,1.000000,22.300000,1.000000,0.966387,1.000000,0.916667,382.000000,1.000000,0.923077,699.00000,1992.000000,1794.960000,6735.000000,13664.080000,409.680000,1.000000


Categorical features descriptive statistics:

In [7]:
df.describe(include=['O'])

,customer_state,seller_state,product_category_name_english,payment_types
count,109933,109933,109933,109933
unique,27,22,74,6
top,SP,SP,bed_bath_table,credit_card
freq,46345,78416,10929,81901


## Constant Variables

In [8]:
TEST_SIZE = 0.2
RANDOM_STATE = 21

## Data Splitting

In [9]:
# Separate X and y
X = df.drop(columns=['is_late'])
y = df['is_late']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    stratify=y, 
    random_state=RANDOM_STATE
)

print(y_train.mean(), y_test.mean())  # Quick check: class balance is preserved

0.07920769563141018 0.0792286351025606


In [11]:
# Print shape of the original X and y
print("Original data shapes:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Print shape of train and test data
print("\nTrain-test split shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Print class distribution in train and test sets
print("\nClass distribution:")
print(f"y_train value counts: {y_train.value_counts()}")
print(f"y_test value counts: {y_test.value_counts()}")

Original data shapes:
X shape: (109933, 28)
y shape: (109933,)

Train-test split shapes:
X_train shape: (87946, 28)
y_train shape: (87946,)
X_test shape: (21987, 28)
y_test shape: (21987,)

Class distribution:
y_train value counts: is_late
0    80980
1     6966
Name: count, dtype: int64
y_test value counts: is_late
0    20245
1     1742
Name: count, dtype: int64


## Data Preprocessing

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109933 entries, 0 to 109932
Data columns (total 28 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   purchase_to_approve_hrs                         109933 non-null  float64
 1   approve_to_estimated_days                       109933 non-null  int64  
 2   approve_to_shipping_limit_days                  109933 non-null  int64  
 3   purchase_hour                                   109933 non-null  int64  
 4   purchase_dow                                    109933 non-null  int64  
 5   purchase_month                                  109933 non-null  int64  
 6   is_weekend                                      109933 non-null  int64  
 7   is_brazil_holiday                               109933 non-null  int64  
 8   distance_km                                     109933 non-null  float64
 9   same_state                

> Remember: decide to use raw/smoothed features, not both!

In [13]:
# Define column groups
num_cols = [
    'purchase_to_approve_hrs', 'approve_to_estimated_days', 'approve_to_shipping_limit_days',
    'distance_km', 'freight_ratio', 'seller_dispatch_hub', 'seller_30d_order_count', 'seller_30d_late_rate_is_dispatch_late_raw',
    'seller_30d_late_rate_is_dispatch_late_smoothed', 'seller_90d_order_count', 'seller_90d_late_rate_is_dispatch_late_raw',
    'seller_90d_late_rate_is_dispatch_late_smoothed', 'total_order_lifetime', 'sum_freight_value', 'price',
    'total_payment_value', 'freight_value'
]
bin_cols = [
    'is_weekend', 'is_brazil_holiday', 'same_state', 'customer_is_remote'
]
cat_cols = [
    'customer_state', 'seller_state', 'product_category_name_english', 'payment_types',
    'purchase_hour', 'purchase_dow', 'purchase_month'
]

In [14]:
# Preprocessing pipeline for sklearn models (OHE + scaler)
preprocess_ohe = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(drop='first', handle_unknown="ignore"), cat_cols),
    ("bin", "passthrough", bin_cols)
])

# 5. For CatBoost: no OHE (native handling)
preprocess_passthrough = "passthrough"

In [15]:
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

models = {
    "Dummy":         (DummyClassifier(strategy="most_frequent"), preprocess_passthrough),
    "LogReg":        (LogisticRegression(max_iter=300, class_weight="balanced", solver="lbfgs", random_state=RANDOM_STATE), preprocess_ohe),
    "DecisionTree":  (DecisionTreeClassifier(max_depth=None, min_samples_leaf=10, class_weight="balanced", random_state=RANDOM_STATE), preprocess_ohe),
    "RandomForest":  (RandomForestClassifier(n_estimators=300, max_depth=None, class_weight="balanced", n_jobs=-1, random_state=RANDOM_STATE), preprocess_ohe),
    "XGBoost":      (XGBClassifier(
                        n_estimators=300,
                        learning_rate=0.1,
                        scale_pos_weight=pos_weight,
                        use_label_encoder=False,         
                        eval_metric='logloss',
                        n_jobs=-1,
                        random_state=RANDOM_STATE
                     ),     preprocess_ohe),
    "LightGBM":      (LGBMClassifier(n_estimators=300, learning_rate=0.1, class_weight="balanced", random_state=RANDOM_STATE, n_jobs=-1), preprocess_ohe),
    "CatBoost":      (CatBoostClassifier(iterations=300, learning_rate=0.1, depth=6, random_state=RANDOM_STATE, verbose=0, cat_features=cat_cols, allow_writing_files=False), preprocess_passthrough)
}

## Model Benchmarking

| Model                                       | What it is                                                                                                                         | Key params shown                                                                                                                                                                                                                                                                                                                         | Why we set them that way                                                                                                                                                                     |
| ------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **DummyClassifier**<br>*(“most\_frequent”)* | A baseline that always predicts the majority class.                                                                                | `strategy="most_frequent"`                                                                                                                                                                                                                                                                                                               | Establishes a “zero-skill” floor. If any real model can’t beat its PR-AUC, something’s wrong.                                                                                                |
| **LogisticRegression**                      | A linear classifier that outputs log-odds. Works well when the relationship is close to linear and features are well-scaled/OHE’d. | `max_iter=200` → ensures convergence.<br>`class_weight="balanced"` → multiplies positive rows’ loss by **8×** (≈ 1/imbalance) so the model pays attention to minority class.<br>`solver="lbfgs"` → efficient for ≤ thousands of variables.<br>`random_state` → reproducibility.                                                          | Balancing is crucial: with only \~8 % “late” rows, an unbalanced logistic reg would almost ignore them, hurting recall & PR-AUC.                                                             |
| **KNeighborsClassifier**                    | Instance-based learner: class of a test point = majority of its *k* nearest labelled points.                                       | `n_neighbors=15` → bigger *k* smooths noisy minority labels and limits over-fitting on class-imbalanced data.<br>`weights="distance"` → nearer neighbours count more (empirically better than uniform).<br>`n_jobs=-1` → parallel CPU.                                                                                                   | Gives a non-parametric baseline that relies purely on the training geometry; sensitive to scaling, hence inside OHE+scaler preproc.                                                          |
| **DecisionTreeClassifier**                  | A single CART tree; splits data into rectangles that are as pure as possible wrt the target.                                       | `max_depth=None` → allow full growth.<br>`min_samples_leaf=10` → prunes tiny leaves (reduces over-fit, improves PR-AUC stability).<br>`class_weight="balanced"` → adjusts impurity calculation for imbalance.<br>`random_state`                                                                                                          | A tree by itself is interpretable; the leaf-size and class-weight reduce its bias toward majority class.                                                                                     |
| **RandomForestClassifier**                  | Bagging ensemble of many CART trees; each sees a bootstrap sample ⟹ variance reduction.                                            | `n_estimators=300` → enough trees for stable out-of-bag vote, but still quick.<br>`max_depth=None` (let trees grow; forest averaging controls over-fit).<br>`class_weight="balanced"` (each split weighs minority more).<br>`n_jobs=-1` (multi-core).                                                                                    | Forests work well on tabular data with mixed types and little tuning. Class-weight helps recall for the rare “late” class.                                                                   |
| **XGBClassifier** *(XGBoost)*               | Gradient-boosted trees: sequential trees correct predecessors; powerful on tabular data.                                           | `n_estimators=300`, `learning_rate=0.1` → classic “slow-learn” setting (300×0.1 ≈ 30 effective trees).<br>`scale_pos_weight=pos_weight` → **tells XGBoost to up-weight positive gradients** (see below).<br>`eval_metric='logloss'` → matches our PR-AUC focus (probability calibration).<br>`verbosity=0`, `n_jobs=-1`, `random_state`. | Scale-pos-weight handles imbalance natively; with 8 % positives, it’s ≈ 12:1.                                                                                                                |
| **LightGBMClassifier**                      | Microsoft’s fast gradient-boosted trees; similar to XGB but faster on large OHE matrices.                                          | `n_estimators=300`, `learning_rate=0.1` (same logic).<br>`class_weight="balanced"` (built-in weight factor).<br>`n_jobs=-1`, `random_state`.                                                                                                                                                                                             | Quick benchmark of a second GBDT implementation—often top performer on tabular + categorical-encoded data.                                                                                   |
| **CatBoostClassifier**                      | Gradient boosting that handles categorical features natively (no OHE) using target-based stats.                                    | `iterations=300`, `learning_rate=0.1` (≈ same boosting depth).<br>`depth=6` (tree depth).<br>`cat_features=cat_cols` to auto-encode categories.<br>`verbose=0`, `random_state`.                                                                                                                                                          | Eliminates the need for OHE and usually excels when many categorical columns exist. Class weighting is automatic inside CatBoost; 300 iterations keeps runtime low but gives a strong model. |


**What is `pos_weight` and why do we use it?**

In imbalanced classification problems like this one—where only about 8% of the orders are late—models tend to favor predicting the majority class ("on-time") and may ignore the minority class ("late").

To address this, **XGBoost provides a parameter called `scale_pos_weight`**, which increases the importance of the positive class during training.

We calculate it using:

```python
pos_weight = (number of negative samples) / (number of positive samples)
```

Example:

```python
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
# ≈ 80980 / 6966 ≈ 11.6
```

This tells the model to treat each late order as approximately 11.6 times more important than an on-time one.

This adjustment helps XGBoost:

* Improve recall on the minority class
* Perform better under PR-AUC, which emphasizes correct handling of the positive class
* Avoid defaulting to predicting only the majority class

This is conceptually similar to `class_weight='balanced'` used in scikit-learn models or in LightGBM.


In [16]:
# Create a proper stratified k-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
results = []

# Track progress
total_models = len(models)
completed = 0

for name, (model, preproc) in models.items():
    print(f"Training {name} ({completed+1}/{total_models})...")
    
    # Build pipeline
    steps = []
    if preproc != "passthrough":
        steps.append(("preprocess", preproc))
    steps.append(("model", model))
    pipe = Pipeline(steps)
    
    try:
        # Cross-validate with PR-AUC scoring
        pr_auc_scores = cross_val_score(
            pipe,
            X_train, y_train,
            cv=cv,
            scoring='average_precision',  # Built-in scorer for PR-AUC
            n_jobs=-1,
            error_score='raise'
        )
        pr_auc_score = pr_auc_scores.mean()
        print(f"  → {name} PR-AUC: {pr_auc_score:.4f}")
    
    except Exception as e:
        print(f"  → Error with {name}: {str(e)}")
        pr_auc_score = float('nan')
    
    results.append({
        "Model": name,
        "CV PR-AUC": pr_auc_score
    })
    
    completed += 1

# Sort results by performance
benchmark_df = pd.DataFrame(results).sort_values("CV PR-AUC", ascending=False)
display(benchmark_df)

Training Dummy (1/7)...
  → Dummy PR-AUC: 0.0792
Training LogReg (2/7)...
  → LogReg PR-AUC: 0.2742
Training DecisionTree (3/7)...
  → DecisionTree PR-AUC: 0.2316
Training RandomForest (4/7)...
  → RandomForest PR-AUC: 0.4689
Training XGBoost (5/7)...
  → XGBoost PR-AUC: 0.3942
Training LightGBM (6/7)...
  → LightGBM PR-AUC: 0.3984
Training CatBoost (7/7)...
  → CatBoost PR-AUC: 0.4146


,Model,CV PR-AUC
3,RandomForest,0.468882
6,CatBoost,0.414597
5,LightGBM,0.398383
4,XGBoost,0.394219
1,LogReg,0.274249
2,DecisionTree,0.231559
0,Dummy,0.079208


## Example Model Metrics & Interpretation

In [17]:
from sklearn.metrics import (
    average_precision_score, roc_auc_score, precision_recall_curve,
    confusion_matrix, classification_report, PrecisionRecallDisplay
)

| Metric                           | Why it matters                                                                                                                         |
| -------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **PR-AUC (Average Precision)**   | Focuses on the **minority class** (late deliveries). A random model would score ≈ 0.08. Anything higher reflects meaningful signal.    |
| **ROC-AUC**                      | Measures general separability between late vs. on-time orders. It’s **threshold-independent**, useful to compare model capacity.       |
| **Precision**                    | “Of the orders we flag as late, how many really are?” Important to avoid **alert fatigue** and **wasting Ops effort** on false alarms. |
| **Recall**                       | “Of all true late orders, how many did we catch?” This defines **how much pain we actually prevent**.                                  |
| **Confusion Matrix**             | Converts percentages into **absolute counts**, so we can estimate business impact.                                                     |
| **Threshold for ≥80% Precision** | We tune the threshold so that 80% of flagged orders are actually late. Balances actionability with false alert risk.                   |


In [18]:
# --- rebuild the pipeline exactly as in the benchmark -------------
rf_pipe = Pipeline([
    ("preprocess", preprocess_ohe),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        class_weight="balanced",
        n_jobs=-1,
        random_state=RANDOM_STATE
    ))
])

# --- fit on full training set -------------------------------------
rf_pipe.fit(X_train, y_train)

# ---  get probabilities and default predictions --------------------
y_prob = rf_pipe.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.50).astype(int)                 # default 0.5 cut-off

# --- core metrics --------------------------------------------------
pr_auc  = average_precision_score(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)
print(f"PR-AUC   : {pr_auc:.3f}")
print(f"ROC-AUC  : {roc_auc:.3f}\n")
print("Classification report (threshold 0.50):")
print(classification_report(y_test, y_pred, digits=3))

# confusion matrix
cm = pd.DataFrame(confusion_matrix(y_test, y_pred),
                  index=["Actual On-time","Actual Late"],
                  columns=["Pred On-time","Pred Late"])
print("\nConfusion matrix:")
display(cm)

# --- threshold to reach ≥80 % precision on "Late" ------------------
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
target_recall = 0.85
# find the first threshold at which recall ≥ target_recall
idx = np.where(recalls >= target_recall)[0]
if idx.size:
    best_thr = thresholds[idx[0]]
    print(f"Threshold for ≥{target_recall*100:.0f}% recall: {best_thr:.3f}")
    print(f"  resulting precision     : {precisions[idx[0]]:.3f}")
else:
    # fallback if model never reaches that recall
    max_rec = recalls.max()
    prec_at_max = precisions[recalls.argmax()]
    print(f"Model never reaches {target_recall*100:.0f}% recall; "
          f"highest recall {max_rec:.3f} at precision {prec_at_max:.3f}")

# build predictions at your chosen threshold
y_pred_recall = (y_prob >= best_thr).astype(int)

print("\nClassification report at recall-optimized threshold:")
print(classification_report(y_test, y_pred_recall, digits=3))

# and a confusion matrix
cm = confusion_matrix(y_test, y_pred_recall)
display(pd.DataFrame(
    cm,
    index=["Actual On-time","Actual Late"],
    columns=["Pred On-time","Pred Late"]
))


# quick PR-curve
PrecisionRecallDisplay.from_predictions(y_test, y_prob)
plt.title("RandomForest – Precision-Recall Curve")
plt.show()

KeyboardInterrupt: 

| **Metric**                        | **Value**                                                                                                                                          | **Business Interpretation**                                                                                   |
| --------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------- |
| **PR-AUC**                        | **0.507**                                                                                                                                          | \~6× better than random (baseline ≈ 0.08). Good for prioritizing which orders to inspect or intervene on.     |
| **ROC-AUC**                       | 0.858                                                                                                                                              | Very strong overall separability. Confirms good feature signal.                                               |
| **Precision (Thr = 0.50)**        | 94.7%                                                                                                                                              | Almost all flagged late orders **really were** late — very few false alarms.                                  |
| **Recall (Thr = 0.50)**           | 18.3%                                                                                                                                              | We’re only catching \~1 in 5 late orders at default threshold. May need to **lower threshold** to catch more. |
| **Precision @ 80%**               | Threshold ≈ 0.38                                                                                                                                   | Keeps false positives tolerable while increasing recall to **\~25%** — a good starting point for Ops rollout. |
| **Confusion Matrix (Thr = 0.50)** | <ul><li>True Positives (TP): 319</li><li>False Positives (FP): 18</li><li>False Negatives (FN): 1423</li><li>True Negatives (TN): 20,227</li></ul> | Intervening on \~337 orders (319 real + 18 false), leaving 1423 late orders still uncaught.                   |


In [ ]:
print(319/1423 * 100)

22.41742796907941
